## Actividades de modificación

### Regularización

MLP con Dropout

In [ ]:
class MLPDropout(nn.Module):
	def __init__(self, input_size=64*64*3, num_classes=10, p=0.5):
		super().__init__()
		self.model = nn.Sequential(
			nn.Flatten(),
			nn.Linear(input_size, 512),
			nn.ReLU(),
			nn.Dropout(p),
			nn.Linear(512, 128),
			nn.ReLU(),
			nn.Dropout(p),
			nn.Linear(128, num_classes)
		)

	def forward(self, x):
		return self.model(x)

MLP con BatchNorm

In [ ]:
class MLPBatchNorm(nn.Module):
	def __init__(self, input_size=64*64*3, num_classes=10):
		super().__init__()
		self.model = nn.Sequential(
			nn.Flatten(),
			nn.Linear(input_size, 512),
			nn.BatchNorm1d(512),
			nn.ReLU(),
			nn.Linear(512, 128),
			nn.BatchNorm1d(128),
			nn.ReLU(),
			nn.Linear(128, num_classes)
		)

	def forward(self, x):
		return self.model(x)

MLP con BatchNorm y Dropout

In [ ]:
class MLPBatchNormDropout(nn.Module):
	def __init__(self, input_size=64*64*3, num_classes=10, p=0.5):
		super().__init__()
		self.model = nn.Sequential(
			nn.Flatten(),
			nn.Linear(input_size, 512),
			nn.BatchNorm1d(512),
			nn.ReLU(),
			nn.Dropout(p),
			nn.Linear(512, 128),
			nn.BatchNorm1d(128),
			nn.ReLU(),
			nn.Dropout(p),
			nn.Linear(128, num_classes)
		)

	def forward(self, x):
		return self.model(x)

# Hay que agregar:
# mlflow.log_param("batchnorm", True)
# mlflow.log_param("dropout_p", 0.5)

Optimizer con Weight Decay (L2)

In [ ]:
optimizer_adv = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

# Hay que agregar:
# mlflow.log_param("weight_decay", 1e-4)

Data Augmentation avanzado

In [ ]:
train_transform_adv = A.Compose([
	A.Resize(64, 64),
	A.HorizontalFlip(p=0.5),
	# A.VerticalFlip(p=0.5),
	A.RandomBrightnessContrast(p=0.2),
	A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
	# A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
	A.Normalize(),
	ToTensorV2()
])

train_dataset_adv = CustomImageDataset(train_dir, transform=train_transform_adv)
train_loader_adv = DataLoader(train_dataset_adv, batch_size=batch_size, shuffle=True)

Early Stopping

In [ ]:
best_val_loss = float('inf')
patience = 5
counter = 0

for epoch in range(n_epochs):
	train_and_validate(model, train_loader, val_loader, criterion, optimizer, n_epochs=10, batch_size=batch_size)
	val_loss, val_acc = evaluate(model, val_loader, epoch=epoch, prefix="val")
	if val_loss < best_val_loss:
		best_val_loss = val_loss
		counter = 0
		torch.save(model.state_dict(), "best_model.pth")
	else:
		counter += 1
		if counter >= patience:
			print(f"Early stopping en epoch {epoch+1}")
			break

### Inicialización de Parámetros

In [ ]:
# Inicialización manual de pesos
class MLPInit(nn.Module):
	def __init__(self, input_size=64*64*3, num_classes=10, init_type='he'):
		super().__init__()
		self.model = nn.Sequential(
			nn.Flatten(),
			nn.Linear(input_size, 512),
			nn.ReLU(),
			nn.Linear(512, 128),
			nn.ReLU(),
			nn.Linear(128, num_classes)
		)
		self.init_weights(init_type)

	def forward(self, x):
		return self.model(x)

	def init_weights(self, init_type):
		for m in self.modules():
			if isinstance(m, nn.Linear):
				if init_type == 'he':
					nn.init.kaiming_normal_(m.weight)
				elif init_type == 'xavier':
					nn.init.xavier_uniform_(m.weight)
				if m.bias is not None:
					nn.init.zeros_(m.bias)

# Hay que agregar:
# mlflow.log_param("init_type", "he")

In [ ]:
# Visualización de pesos
for name, param in model.named_parameters():
	if 'weight' in name:
		writer.add_histogram(name, param, epoch)

## Pruebas

In [ ]:
variantes = [
	"base",
	# "pocas_imagenes",
	# "grises",
	"dropout",
	"batchnorm",
	"batchnorm_dropout",
	"weight_decay",
	"augmentation",
	"init_he",
	"init_xavier",
	"histogramas"
]

for var in variantes:
	print(f"Entrenando variante: {var}")
	
	if var == "base" or var == "weight_decay" or var == "augmentation" or var == "histogramas":
		model = MLPClassifier(num_classes=num_classes).to(device)
	# elif var == "grises":
	# elif var == "pocas_imagenes":
	elif var == "init_he":
		model = MLPInit(init_type='he', num_classes=num_classes).to(device)
	elif var == "init_xavier":
		model = MLPInit(init_type='xavier', num_classes=num_classes).to(device)
	elif var == "dropout":
		model = MLPDropout(num_classes=num_classes).to(device)
	elif var == "batchnorm":
		model = MLPBatchNorm(num_classes=num_classes).to(device)
	elif var == "batchnorm_dropout":
		model = MLPBatchNormDropout(num_classes=num_classes).to(device)
	else:
		raise ValueError(f"Variante '{var}' no reconocida.")

	if var == "weight_decay":
		optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
	else:
		optimizer = optim.Adam(model.parameters(), lr=1e-3)
	if var == "augmentation":
		train_loader = train_loader_adv

	writer = SummaryWriter(log_dir=f"runs/experimento_{var}")
	
	for epoch in range(n_epochs):
		# Train...
		# writer.add_scalar(...)
		if var == "histogramas":
			# Histogramas de pesos
			for name, param in model.named_parameters():
				if 'weight' in name:
					writer.add_histogram(name, param, epoch)
	
	torch.save(model.state_dict(), f"mlp_{var}.pth")
	mlflow.log_param("variante", var)
	mlflow.log_artifact(f"mlp_{var}.pth")
	
	writer.close()
	print(f"Variante {var} entrenada y guardada.\n")

In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt
# results = pd.read_csv("resultados_experimentos.csv")
# display(results.sort_values("val_accuracy", ascending=False))
# variantes = ["Base", "Dropout", "BatchNorm", "Dropout+BatchNorm", "Init He", "Init Xavier", "L2", "Augment", "Grises"]
# plt.figure(figsize=(10,5))
# plt.bar(variantes, results.sort_values("val_accuracy", ascending=False)["val_accuracy"], color='skyblue')
# plt.ylabel("Validation Accuracy")
# plt.title("Comparación de variantes MLP")
# plt.xticks(rotation=45)
# plt.show()